In [141]:
import re

In [142]:
class Node:
    def __init__(self, network, x, y, text):
        self.index = -1
        self.network = network
        self.x = x
        self.y = y
        self.text = text
        self.links = []
        
        self.network.add_node(self)
        
    def __str__(self):
        return f"[{self.text}]"
    
    def add_link(self, link):
        self.links.append(link)

class Link:
    def __init__(self, network, from_node, to_node, cost):
        self.network = network
        self.from_node = from_node
        self.to_node = to_node
        self.cost = cost
    
        self.network.add_link(self)
        self.from_node.add_link(self)
        
    def __str__(self):
        return f"[{self.from_node.text}] --> [{self.to_node.text}] ({self.cost})"

class Network:
    def __init__(self):
        self.clear()
    
    def clear(self):
        self.nodes = []
        self.links = []
        
    def add_node(self, node: Node):
        node.index = len(self.nodes)
        self.nodes.append(node)
        
    def add_link(self, link: Link):
        self.links.append(link)
        
    def to_string(self):
        return (
            "{} # Num nodes.".format(len(self.nodes)) +
            "\n{} # Num links.".format(len(self.links)) +
            "\n# Nodes." +
            "\n{}".format('\n'.join([f"{n.x},{n.y},{n.text}" for n in self.nodes])) +
            "\n# Links." +
            "\n{}".format('\n'.join([f"{l.from_node.index},{l.to_node.index},{l.cost}" for l in self.links]))
        )
    
    def save_into_file(self, file_name):
        open(file_name, 'w').write(self.to_string())
    
    @staticmethod
    def read_next_line(file_handle):
        while line := file_handle.readline():
            pattern = re.compile('([^#]*)(#.*)?$')
            matcher = pattern.match(line)
            line = matcher.group(1).strip() if matcher else None
            if line:
                return line 
        return None
    
    def load_from_file(self, file_name):
        self.clear()
        
        with open(file_name, 'r') as file_handle:
            num_nodes = int(self.read_next_line(file_handle))
            num_links = int(self.read_next_line(file_handle))
            # get nodes
            while num_nodes > 0 and (line := self.read_next_line(file_handle)):
                num_nodes -= 1
                if m := re.match(r'(\d+),(\d+),(.*)', line):
                    x = int(m.group(1))
                    y = int(m.group(2))
                    text = m.group(3)
                    Node(self, x, y, text)
            # get links
            while num_links > 0 and (line := self.read_next_line(file_handle)):
                num_links -= 1
                if m := re.match(r'(\d+),(\d+),(\d+)', line):
                    from_node_idx = int(m.group(1))
                    to_node_idx = int(m.group(2))
                    cost = int(m.group(3))
                    Link(self, self.nodes[from_node_idx], self.nodes[to_node_idx], cost)


In [143]:
def make_test_network(file_name):
    # Build a tiny test network.
    network = Network()
    a = Node(network,  20,  20, 'A')
    b = Node(network, 120, 120, 'B')
    
    link_a_b = Link(network, a, b, 10)
    
    # Save into a test file.
    network.save_into_file(file_name)


In [144]:
file_name = '3nodes.net'
make_test_network(file_name)

network = Network()
network.load_from_file(file_name)

network.save_into_file('copy_of_3nodes.net')
